## Welcome to the National Water Model (NWM) Sandbox 2! 

<strong><em>Created by <a href="https://www.linkedin.com/in/justin-hunter-0b86871a6/" target="_blank">Justin Hunter</a>, <a href="https://www.linkedin.com/in/danames/" target="_blank">Dr. Dan Ames</a>, and <a href="https://www.linkedin.com/in/easton-perkins-02968a156/" target="_blank">Easton Perkins</a>.</em></strong><br>
<em><strong>June, 2021. Brigham Young University. Provo, Utah.
<a href="hydroinformatics.byu.edu" target="_blank">BYU Hydroinformatics Lab</a>.</em></strong>

In this Jupyter Notebook we will take a look at  a NetCDF (Network Common Data Form) file obtained from the NWM using a Python package called xarray. NetCDF files are a standardized way of exchanging scientific data. NetCDF is well suited for multidimensional datasets containing meteorological or observational data. NetCDF files work well for NWM forecasts and contain a lot of useful metadata. xarray is a Python package which is built on NumPy and pandas and works well with NetCDF. We will also explore more ways to visualize the national water model and display its reaches on a map. 

<em><h4>Imports</h4></em>

The next cell installs or imports some Python modules or packages that will be used in this notebook. A brief explanation of what each one does is included below:
* xarray makes it easier to work with multidimensional datasets like the NWM forecasts. 
* Importing the date type from the datetime module allows us to call todays date. 
* The os module allows us to communicate with the operating system. 
* pandas is a useful library for data manipulation and analysis.
* ipywidgets will be used to select a forecast type from a dropdown menu later on.
* The requests module lets us make requests to web pages. 
* matplotlib will help us create plots.
* ipyleaflet is a package for creating interactive maps in Python.

In [ ]:
import xarray as xr
import numpy as np
from datetime import date
import os
import pandas as pd
import ipywidgets as widgets

import requests
from requests import Request

import matplotlib.pyplot as plt
%matplotlib inline

import ipyleaflet
from ipyleaflet import Map, WMSLayer, basemaps, LayersControl, DrawControl, Popup, Marker

<em><h4>Functions</em></h4>

This next cell should look familiar! It defines the functions that were used in the first sandbox. These functions build the appropriate url for a NWM forecast, get the NetCDF file stored at that url, and then build a time series of data for a given reach or stream.

In [ ]:
def GetForecastFileName(ForecastStartDate = '20210321', ForecastStartTimestep='00', ForecastType = 'short_range', ForecastMember='1', TimeStep = '001'):
  BaseName = 'https://nomads.ncep.noaa.gov/pub/data/nccf/com/nwm/prod/nwm.'

  ForecastStartDate

  if (ForecastType == 'short_range'):
    return BaseName + ForecastStartDate + '/short_range/nwm.t' + ForecastStartTimestep +'z.short_range.channel_rt.f' + TimeStep + '.conus.nc'
  elif (ForecastType == 'medium_range'): 
    return BaseName + ForecastStartDate + '/medium_range_mem' + ForecastMember + '/nwm.t' + ForecastStartTimestep +'z.medium_range.channel_rt_' + ForecastMember + '.f' + TimeStep + '.conus.nc'
  elif (ForecastType == 'long_range'):
    return BaseName + ForecastStartDate + '/long_range_mem' + ForecastMember + '/nwm.t' + ForecastStartTimestep +'z.long_range.channel_rt_' + ForecastMember + '.f' + TimeStep + '.conus.nc'
  else:
    return 'error'

def GetForecastFile(Url = 'https://nomads.ncep.noaa.gov/pub/data/nccf/com/nwm/prod/nwm.20210321/short_range/nwm.t00z.short_range.channel_rt.f001.conus.nc'):
  FileName = os.path.basename(Url)
  if os.path.exists(FileName):
    os.remove(FileName)
  r = requests.get(Url, allow_redirects=True)
  open(FileName, 'wb').write(r.content)
  return FileName

def GetSeries(StreamID = 23275226, ForecastStartDate = '20210321', ForecastStartTimestep='00', ForecastType = 'short_range', ForecastMember='1'):
  TimeSteps = []
  TimeSteps.clear()
  Series = []
  Series.clear()
  if (ForecastType=='short_range'):
    for i in range(18):
      TimeSteps.append("%03d" % (i+1))
  elif (ForecastType=='medium_range' and ForecastMember=='1'):
    for i in range(80):
      TimeSteps.append("%03d" % ((i+1)*3))
  elif (ForecastType=='medium_range' and ForecastMember!='1'):
    for i in range (68):
      TimeSteps.append("%03d" % ((i+1)*3))
  elif (ForecastType=='long_range'):
    for i in range(120):
      TimeSteps.append("%03d" % ((i+1)*6))
  else: 
    return 'Error building time steps'
  
  for ts in TimeSteps:
    MyUrl = GetForecastFileName(ForecastStartDate,ForecastStartTimestep, ForecastType,ForecastMember,ts)
    FileName = GetForecastFile(MyUrl)
    if(FileName != 'error'):
      data = xr.open_dataset(FileName)
      Q = float(data.sel(feature_id=StreamID).streamflow.values)
      Series.append(Q)
    else:
      print('Error getting forecast files.')
    
  return Series

<em><h4>Variables</em></h4>

Next let's define some variables that we can use to download a forecast of our choice. Then we'll get the url and download the corresponding NetCDF forecast file.

In [ ]:
# These variables should look familiar to you from the first sandbox
ForecastStart = '00'
Type = 'short_range'
Member = '1'
Timestep = '001'
today = date.today()
today = str(today)
today = today.replace("-", "")

url = GetForecastFileName(today, ForecastStart,Type,Member,Timestep)
path = GetForecastFile(url)

The next cell creates a variable called 'ds' and assigns it a value of the given NetCDF file opened as an xarray dataset. That file was called in using a variable called 'path' which was defined in the previous cell. Take a look at the output of the next cell and try to learn a little bit about the metadata included with this dataset. You'll be able to see the different dataset dimensions, coordinates, variables, and attributes of the dataset.

<em><h4>xarray</em></h4>

In [ ]:
ds = xr.open_dataset(path)
ds

xarray is great for analyzing multidimensional data such as a NWM forecast. It has great metadata support and is always the best option when working with NetCDF. For more information about xarray visit: http://xarray.pydata.org/en/stable/index.html.

<em><h4>Plot</em></h4>

Now let's try to visualize 'ds' using matplotlib. 

In [ ]:
plt.rcParams['figure.figsize'] = (8,6)
ds.streamflow.plot()

That plot isn't great. Because each NetCDF file contains a certain forecast (short, medium, long, etc.) at a certain time for the entire NWM network (every single reach covered by the NWM), the plot is displaying Reach IDs on the x axis and streamflow on the y. This isn't very helpful. Let's try instead to create a map that will display all of the NWM reaches and let us see some of their attributes.

<em><h4>Mapping</em></h4>

First we should define some variables that will help us create our map.

In [ ]:
# zoom level
zoom = 4
# lat and long coordinates for the center of the map
center = (39,-96)

Next, we need to create our map with ipyleaflet and then add a WMS layer too it. WMS (Web Map Services) allows us to pull in data that is stored somewhere on the web. In this case the data is coming from a shapefile stored in a Hydroshare resource. Hydroshare is a web based hydrologic information system for sharing and publishing hydrologic data. Here we only pull in one layer which contains the NWM reaches lying inside of Hydrologic Unit Code (HUC) number 1. The Hydroshare resource id is 'HS-f5fa9306f92147918fc500c386cf0dd9' and the shapefile is named 'HUC1'. 

The resource is located here: https://www.hydroshare.org/resource/f5fa9306f92147918fc500c386cf0dd9/. 

That particular resource only contains a few HUCs. There are 18 in total. Several Hydroshare resources make up a collection which contains all of the NWM reaches divided up by HUC. 

The collection can be found here: https://www.hydroshare.org/resource/c16596e525bf41e2ae843f1e3bbcef90/. 

In [ ]:
# Create a map called 'mappy' with a basemap, and the zoom level and center that we defined previously.
mappy = Map(basemap=basemaps.CartoDB.Positron, center=center, zoom=zoom)

In [ ]:
# Create a WMS Layer from the desired Hydroshare resource and desired layer (shapefile).
wmslay = WMSLayer(
    url = 'https://geoserver.hydroshare.org/geoserver/HS-f5fa9306f92147918fc500c386cf0dd9/wms?',
    layers = 'HS-f5fa9306f92147918fc500c386cf0dd9:HUC1',
    transparent=True,
    name = 'HUC1',
    format = 'image/png'
)

# Create a layers control at the top right of the map
control = LayersControl(position='topright')

# Add the layers control to 'mappy'
mappy.add_control(control)

# Add the WMS Layer to 'mappy'
mappy.add_layer(wmslay)

In [ ]:
# Show 'mappy'
mappy

This map shows all of the NWM reaches in HUC1, but we haven't actually done anything which would allow us to view the forecasts yet.

Next we are going to use some Jupyter notebook magic to run HTML and display a web map. What you see in the next cell is HTML code. All it really does is display the web page at https://justinhunter1.github.io/csb-yn7y5/. 

This web page was built using javascript to display all of the NWM reaches and their attributes. It uses a WMS layer containing all of the NWM Reaches and is built using openlayers which is a javascript library for mapping. 

Zooming in to a specific part of the map will prompt the reaches to appear. By clicking on a specific reach you can then obtain its objectid or unique identifier. 

The source code for this map and web page can be found here: https://github.com/justinhunter1/NWM_map_ForSandbox2. It consists of several files of code, but the important stuff is primarily in the main.js file. In less than 100 lines of code you could create a similar map yourself!

In [ ]:
%%HTML
<html>
<head></head>
<body>
<iframe title="NWM Reaches" src="https://justinhunter1.github.io/csb-yn7y5/" width=900 height=600>
</iframe>
</body>
</html>

After you've had a chance to play around and obtain an object id for a stream of interest, run the next cell and enter in the object id when prompted. Make sure you are getting the value from the 'OBJECTID' cell and not the 'station_id' value.

In [ ]:
# Running this cell will prompt the user to enter in a value which will then be assigned to the variable 'reach'
reach = input("Reach/Stream/Feature ID: ")

In [ ]:
# Convert 'reach' from string type to integer type
reach = int(reach)

In [ ]:
# Here we use ipywidgets to create a menu dropdown with options for the NWM forecast types.
menu = widgets.Dropdown(
       options=['short_range', 'medium_range', 'long_range'],
       value='short_range',
       description='Forecast:')

In [ ]:
# This cell will display the menu that we created in the last cell. Select your desired forecast from it.
menu

In [ ]:
# Now lets build a time series for the reach you entered and the forecast type selected from 'menu'.
Series = GetSeries(reach, today, '00', menu.value, '1')
print(Series)

In [ ]:
# Now lets create a plot of the forecast like we did in the first sandbox.
plt.rc('font', size=14)
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(Series, color='tab:blue', label='Streamflow')
ax.set_xlabel('Time')
ax.set_ylabel('Flowrate (cms)')
ax.set_title(menu.value + ' for Stream ID ' + str(reach))
ax.grid(True)

In [ ]:
# Lastly we'll erase any nwm files leftover in our Jupyter notebook's file browser.
files = [f for f in os.listdir('.') 
         if os.path.isfile(f)]

for f in files:
    # Look at every file and if contains 'nwm' then remove/delete it! 
    if "nwm" in f:
        os.remove(f)

Thank you for using this resource! Hopefully this exercise has helped open your eyes to what can be done to analyze NWM forecasts and create maps and plots of the NWM network.